### Pre-requisites:

In [1]:
#Libraries for time computation and data importation:
import time
import os

#Libraries to manage the Interaction Matrix:
import pandas as pd
import numpy as np

#Utility Functions:
import utils
from utils import *

In [2]:
#Data Structures:

import DataStructures
from DataStructures import *

### Algorithms:

In [3]:
#Algorithm to select the film preferred by a user:
def search_favourite(data, target, unseen):
    """
    FUNCTION:
    Return the favourite items for a target user.
    
    PARAMETERS:
    data -> Pandas Dataframe with the user-item Interaction Matrix.
    target -> Target user.
    unseen -> Items not rated by the target user.
    
    RETURNS:
    Circular Queue containing the items preferred by the target user.
    """
    #Extract the items rated by the target user and the ratings assigned to them
    valid_i = data.iloc[target].drop(unseen).index
    valid_r = data.iloc[target][valid_i]
    
    #Circular Queue to store the favourite items and their ratings
    favourites = Queue_c(len(valid_i) + 1)
    
    #Inizialize the Circular Queue
    favourites.enqueue(valid_r[0], valid_i[0])
    best = valid_r[0]
    
    #Search all the favourite items (the ones related to the highest rating)
    for (rating, item) in zip(valid_r, valid_i):
        
        #Check if we find an item with an higher rating
        if rating >= best:
            favourites.enqueue(rating, item)
            best = rating

    #Remove the items related to a lower rating than the actual "best"
    while favourites.first_in()[0] < best:
        favourites.dequeue()
    
    return favourites

def nearest(data, target, favourites, unseen):
    """
    FUNCTION:
    Find the nearest items in the Interaction Matrix for the target user. 
    
    PARAMETERS:
    data -> Pandas Dataframe with the user-item Interaction Matrix.
    target -> Target user.
    favourites -> Circular Queue containing the items preferred by the target user.
    k -> Number of neighbors.
    unseen -> Items not rated by the target user.
    
    RETURNS:
    Dictionary containing the k-nearest items to the target user's preferred items.
    """
    #Dictionary to store the k-neighbors for each favoruite item
    nearest = {}
    
    #Loop through all the favourite items
    while not favourites.is_empty():
        favourite = favourites.dequeue()[1]
        
        #We search only through the items not rated by the user
        neighbors = MaxHeap()
        
        #We search only through the items not rated by the user
        for item in unseen:
            neighbors.insert(cosine(data[item].values, data[favourite].values), item)
        
        #Assign the Priority Queue to its proper favourite item
        nearest[favourite] = neighbors
        
    return nearest

def suggest(target, nearest, unseen, k = 3):
    """
    FUNCTION:
    Returns a Dictionary that links each k-nearest item (keys of the Dictionary) to a class of recommendation. The assignment
    takes care of how many times the item appears as one of the k-nearest neighbors for the target user's favourite items.
    
    PARAMETERS:
    target -> Target user.
    nearest -> Dictionary of Priority Queues containing the target user's neighbors for each favourite item.
    unseen -> Items not rated by the target user.
    k -> Number of neighbors.

    RETURNS:
    Dictionary containing the recomendations for the target user.
    """
    #Dictionary to store each unseen item with its related class of recommendation
    suggestions = {}
    unseen = list(unseen)
    
    #Loop through all the favourite items
    for favourite in nearest.keys():
        iteration = k
        neighbors = nearest[favourite]
        
        #Loop until we've found all the k-neighbors or until the Priority Queue contain some items
        while (iteration > 0) and (neighbors.size > 0):
            suggested = neighbors.delete()[1]
            
            #Rule defined for the recommendation class assignment
            if suggested not in suggestions.keys():
                suggestions[suggested] = "Recommended"
                unseen.remove(suggested)
            else:
                suggestions[suggested] = "Strongly Recommended"
            iteration -= 1
            
    #Rule defined for the recommendation class assignment
    for item in unseen:
        suggestions[item] = "Not Recommended"
    
    return suggestions

def recommend(data, target, k = 3):
    """
    FUNCTION:
    Perform in a single command all the previous algorithms necessary to obtain the recommendation.
    
    PARAMETERS:
    data -> Pandas Dataframe with the user-item Interaction Matrix.
    target -> Target user.
    k -> Number of neighbors.
    
    RETURNS:
    Dictionary containing the recomendations for the target user.
    """
    unseen = data.iloc[target][np.isnan(data.iloc[target])].index
    favourites = search_favourite(data, target)
    neighbors = nearest(data, target, favourites, unseen)
    return suggest(target, nearest, unseen, k = k)

### Test on real data:

In [4]:
#Data importation:
path = os.path.join(os.getcwd(), "data_s.csv")
df = pd.read_csv(path)
df

,Toy Story,Twelve Monkeys,Se7en,The Usual Suspects,Braveheart,Apollo 13,Star Wars,Pulp Fiction,The Shawshank Redemption,Forrest Gump,...,Independence Day,The Godfather,The Empire Strikes Back,Raiders of the Lost Ark,Return of the Jedi,Back to the Future,The Matrix,American Beauty,Fight Club,The Lord of the Rings: The Fellowship of the Ring
0,NaN,NaN,4.0,4.0,4.0,5.0,NaN,4.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.5,5.0,5.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,5.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,5.0,...,NaN,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,...,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,5.0,NaN,5.0,NaN,5.0,3.0,...,3.0,NaN,5.0,5.0,5.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,NaN,4.0,NaN,NaN,NaN,NaN,4.0,4.0,5.0,4.0,...,NaN,NaN,4.0,NaN,4.0,5.0,5.0,3.0,5.0,5.0
485,NaN,NaN,NaN,NaN,2.0,4.0,NaN,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,NaN,5.0,NaN,NaN,4.0,4.0,NaN,5.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,4.0,3.0,5.0,5.0,3.0,3.0,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN


In [5]:
#Test the algorithms on one user:
print("Suggestions for user: " + str(3))
unseen = df.iloc[3][np.isnan(df.iloc[3])].index
favourites = search_favourite(df, 3, unseen)
neighbors = nearest(df, 3, favourites, unseen)
suggest(3, neighbors, unseen)

Suggestions for user: 3


{'Toy Story': 'Strongly Recommended',
 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended',
 'Fight Club': 'Strongly Recommended',
 'Twelve Monkeys': 'Not Recommended',
 'Se7en': 'Not Recommended',
 'The Usual Suspects': 'Not Recommended',
 'Braveheart': 'Not Recommended',
 'Star Wars': 'Not Recommended',
 'Pulp Fiction': 'Not Recommended',
 'The Shawshank Redemption': 'Not Recommended',
 'True Lies': 'Not Recommended',
 'The Fugitive': 'Not Recommended',
 'Jurassic Park': 'Not Recommended',
 "Schindler's List": 'Not Recommended',
 'Terminator 2: Judgment Day': 'Not Recommended',
 'Dances with Wolves': 'Not Recommended',
 'Batman': 'Not Recommended',
 'The Silence of the Lambs': 'Not Recommended',
 'Fargo': 'Not Recommended',
 'Independence Day': 'Not Recommended',
 'The Empire Strikes Back': 'Not Recommended',
 'Raiders of the Lost Ark': 'Not Recommended',
 'Return of the Jedi': 'Not Recommended',
 'Back to the Future': 'Not Recommended',
 'The Matrix': 'Not Re

In [6]:
#Test the algorithms for all the users:
start_time = int(time.ctime().split()[3].split(":")[0])*60*60 + int(time.ctime().split()[3].split(":")[1])*60 + int(time.ctime().split()[3].split(":")[2])

for target in df.index:
    print(target)
    unseen = df.iloc[target][np.isnan(df.iloc[target])].index
    favourites = search_favourite(df, target, unseen)
    neighbors = nearest(df, target, favourites, unseen)
    print(suggest(target, neighbors, unseen))
    print("-"*125)

current = int(time.ctime().split()[3].split(":")[0])*60*60 + int(time.ctime().split()[3].split(":")[1])*60 + int(time.ctime().split()[3].split(":")[2])
print("\nTotal time required: " + str(current - start_time) + "s")

0
{'Toy Story': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'Twelve Monkeys': 'Not Recommended', 'Star Wars': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'True Lies': 'Not Recommended', 'Fargo': 'Not Recommended', 'Independence Day': 'Not Recommended', 'The Godfather': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Raiders of the Lost Ark': 'Not Recommended', 'Return of the Jedi': 'Not Recommended', 'Back to the Future': 'Not Recommended', 'The Matrix': 'Not Recommended', 'American Beauty': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
1
{'Toy Story': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'The Matrix': 'Strongly Recommended', 'Twelve Monkeys': 'Not Recommended', 'Se7en': 'Not Recommended', 'Th

{'Toy Story': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'The Lion King': 'Not Recommended', 'Aladdin': 'Not Recommended', 'Fargo': 'Not Recommended', 'The Matrix': 'Not Recommended', 'American Beauty': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
14
{'Toy Story': 'Strongly Recommended', 'Independence Day': 'Strongly Recommended', 'Fargo': 'Strongly Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'The Lion King': 'Not Recommended', 'True Lies': 'Not Recommended', "Schindler's List": 'Not Recommended', 'Dances with Wolves': 'No

{'Twelve Monkeys': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'Se7en': 'Not Recommended', 'The Usual Suspects': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'Star Wars': 'Not Recommended', 'Pulp Fiction': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'The Lion King': 'Not Recommended', 'True Lies': 'Not Recommended', 'The Fugitive': 'Not Recommended', 'Dances with Wolves': 'Not Recommended', 'Batman': 'Not Recommended', 'The Silence of the Lambs': 'Not Recommended', 'Independence Day': 'Not Recommended', 'The Godfather': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Raiders of the Lost Ark': 'Not Recommended', 'Back to the Future': 'Not Recommended', 'The Matrix': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
30
{'Se7en': 'Strongly Recommended', 'Th

{'Twelve Monkeys': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'Se7en': 'Not Recommended', 'The Usual Suspects': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'True Lies': 'Not Recommended', "Schindler's List": 'Not Recommended', 'Aladdin': 'Not Recommended', 'Fargo': 'Not Recommended', 'The Godfather': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Raiders of the Lost Ark': 'Not Recommended', 'Return of the Jedi': 'Not Recommended', 'Back to the Future': 'Not Recommended', 'The Matrix': 'Not Recommended', 'American Beauty': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
44
{'Twelve Monkeys': 'Recommended', 'Fight Club': 'Recommended', 'American Beauty': 'Recommended', 'Se7en': 'Not Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Pu

{'The Usual Suspects': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'Forrest Gump': 'Not Recommended', 'True Lies': 'Not Recommended', 'The Fugitive': 'Not Recommended', 'The Godfather': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Raiders of the Lost Ark': 'Not Recommended', 'Return of the Jedi': 'Not Recommended', 'Back to the Future': 'Not Recommended', 'The Matrix': 'Not Recommended', 'American Beauty': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
58
{'Se7en': 'Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Recommended', 'Fight Club': 'Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'Pulp Fiction': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'Forrest Gump': 

...

{'Toy Story': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'American Beauty': 'Strongly Recommended', 'Twelve Monkeys': 'Not Recommended', 'Se7en': 'Not Recommended', 'The Usual Suspects': 'Not Recommended', 'Pulp Fiction': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'The Fugitive': 'Not Recommended', 'Dances with Wolves': 'Not Recommended', 'The Silence of the Lambs': 'Not Recommended', 'Fargo': 'Not Recommended', 'The Godfather': 'Not Recommended', 'Back to the Future': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
434
{'Toy Story': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Star Wars': 'Not Recommended', 'The Shawshank Redemption': 'Not Reco

{'Toy Story': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Back to the Future': 'Strongly Recommended', 'Twelve Monkeys': 'Not Recommended', 'Se7en': 'Not Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'Star Wars': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'Forrest Gump': 'Not Recommended', 'The Lion King': 'Not Recommended', 'True Lies': 'Not Recommended', 'The Fugitive': 'Not Recommended', 'Jurassic Park': 'Not Recommended', "Schindler's List": 'Not Recommended', 'Aladdin': 'Not Recommended', 'Terminator 2: Judgment Day': 'Not Recommended', 'Dances with Wolves': 'Not Recommended', 'Batman': 'Not Recommended', 'Fargo': 'Not Recommended', 'Independence Day': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Raiders of the Lost Ark': 'Not Recommended', 'Return of the Jedi': 'Not Recommended'}
---------------------------

{'Toy Story': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'The Matrix': 'Strongly Recommended', 'Twelve Monkeys': 'Not Recommended', 'Se7en': 'Not Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'Pulp Fiction': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'Forrest Gump': 'Not Recommended', 'The Lion King': 'Not Recommended', 'The Fugitive': 'Not Recommended', 'Jurassic Park': 'Not Recommended', "Schindler's List": 'Not Recommended', 'Aladdin': 'Not Recommended', 'Terminator 2: Judgment Day': 'Not Recommended', 'Dances with Wolves': 'Not Recommended', 'Batman': 'Not Recommended', 'Fargo': 'Not Recommended', 'Independence Day': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Back to the Future': 'Not Recommended'}
--------------------------------------------------------------------------------------------------------------

{'Twelve Monkeys': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'Braveheart': 'Not Recommended', 'Apollo 13': 'Not Recommended', 'The Shawshank Redemption': 'Not Recommended', 'Forrest Gump': 'Not Recommended', 'The Lion King': 'Not Recommended', 'True Lies': 'Not Recommended', 'Jurassic Park': 'Not Recommended', 'Aladdin': 'Not Recommended', 'Batman': 'Not Recommended', 'Independence Day': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Raiders of the Lost Ark': 'Not Recommended', 'Return of the Jedi': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
471
{'Toy Story': 'Recommended', 'American Beauty': 'Recommended', 'Back to the Future': 'Recommended', 'Twelve Monkeys': 'Not Recommended', 'The Usual Suspects': 'Not Recommended', 'Braveheart': 'Not Recommended', 'Apollo 13': 'Not R

{'Star Wars': 'Strongly Recommended', 'The Lord of the Rings: The Fellowship of the Ring': 'Strongly Recommended', 'Fight Club': 'Strongly Recommended', 'Pulp Fiction': 'Not Recommended', 'Forrest Gump': 'Not Recommended', 'The Lion King': 'Not Recommended', 'True Lies': 'Not Recommended', 'Jurassic Park': 'Not Recommended', 'Aladdin': 'Not Recommended', 'Terminator 2: Judgment Day': 'Not Recommended', 'Batman': 'Not Recommended', 'Independence Day': 'Not Recommended', 'The Godfather': 'Not Recommended', 'The Empire Strikes Back': 'Not Recommended', 'Raiders of the Lost Ark': 'Not Recommended', 'Return of the Jedi': 'Not Recommended', 'Back to the Future': 'Not Recommended'}
-----------------------------------------------------------------------------------------------------------------------------
488
{'Twelve Monkeys': 'Strongly Recommended', 'Back to the Future': 'Strongly Recommended', 'Return of the Jedi': 'Strongly Recommended', 'Se7en': 'Not Recommended', 'Braveheart': 'Not Reco